In [2]:
import requests

url = "https://www.ah.nl/"

response = requests.get(url)

if response.status_code == 200:
    print("Here is the result:")
    print(f"{response.text[:300]} ...")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


Failed to retrieve the webpage. Status code: 403


In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

def find_ah_product_page(search_term):

    options = Options()
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
    options.add_argument(f'user-agent={user_agent}')

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get("https://www.google.com")

    button = driver.find_element('id', 'W0wltc')
    button.click()

    search_bar = driver.find_element("name", "q")
    search_bar.send_keys(search_term)

    search_bar.send_keys(Keys.RETURN)

    time.sleep(3)

    first_result = driver.find_element(By.CSS_SELECTOR, '#rso > div.hlcw0c > div > div > div > div > div > div > div > div.yuRUbf > div > span > a > h3')
    first_result.click()

    button = driver.find_element('id', 'decline-cookies')
    button.click()

    base_url = driver.current_url
    producten_url = base_url + "producten"
    driver.get(producten_url)

    categories = driver.find_elements(By.CSS_SELECTOR, 'div.product-category-overview_category__vqzcb:nth-child(1) > div:nth-child(1) > a:nth-child(1)')
    for category in categories:
        print(category.text)

    driver.quit()

search_term = "Albert Heijn"
find_ah_product_page(search_term)
    


AttributeError: 'WebDriver' object has no attribute 'find_eements'

In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys



options = Options()
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
options.add_argument(f'user-agent={user_agent}')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.ah.nl/producten")

button = driver.find_element('id', 'decline-cookies')
button.click()

time.sleep(3)





In [19]:
category_urls = []

try:
    category_elements = driver.find_elements(By.CSS_SELECTOR, '.taxonomy-card_title__vBWel a')
    for element in category_elements:
        print(element.text)
        href = element.get_attribute('href')
        category_urls.append(href)
        print(href)
except Exception as e:
    print("Error finding categories:", e)

Aardappel, groente, fruit
https://www.ah.nl/producten/aardappel-groente-fruit
Salades, pizza, maaltijden
https://www.ah.nl/producten/salades-pizza-maaltijden
Vlees, kip, vis, vega
https://www.ah.nl/producten/vlees-kip-vis-vega
Kaas, vleeswaren, tapas
https://www.ah.nl/producten/kaas-vleeswaren-tapas
Zuivel, plantaardig en eieren
https://www.ah.nl/producten/zuivel-plantaardig-en-eieren
Bakkerij en banket
https://www.ah.nl/producten/bakkerij-en-banket
Ontbijtgranen en beleg
https://www.ah.nl/producten/ontbijtgranen-en-beleg
Snoep, koek, chips en chocolade
https://www.ah.nl/producten/snoep-koek-chips-en-chocolade
Tussendoortjes
https://www.ah.nl/producten/tussendoortjes
Frisdrank, sappen, koffie, thee
https://www.ah.nl/producten/frisdrank-sappen-koffie-thee
Wijn en bubbels
https://www.ah.nl/producten/wijn-en-bubbels
Bier en aperitieven
https://www.ah.nl/producten/bier-en-aperitieven
Pasta, rijst en wereldkeuken
https://www.ah.nl/producten/pasta-rijst-en-wereldkeuken
Soepen, sauzen, kruide

In [21]:
category_urls = category_urls[0:16]

category_urls2 = category_urls[5:16]

print(category_urls[1])


https://www.ah.nl/producten/salades-pizza-maaltijden


In [4]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd




def extract_ah_products(driver, category_url):
    producten = []

    
    driver.get(category_url)
    driver.maximize_window()


    button = driver.find_element('id', 'decline-cookies')
    button.click()

    wait = WebDriverWait(driver, timeout = 30)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Filter op Merk: AH"]')))
    try:
        
        ah_filter = driver.find_element(By.CSS_SELECTOR, '[aria-label="Filter op Merk: AH"]')
        ah_filter.click()
        
    except NoSuchElementException:
        print("AH filter not found")
    

    while True:
        try:
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '[data-testhook="load-more"]')))
            meer_resulten = driver.find_element(By.CSS_SELECTOR, '[data-testhook="load-more"]')
            meer_resulten.click()

        except:
            break

    products = driver.find_elements(By.CSS_SELECTOR, '[data-testhook="product-card"]')
    for product in products:
        try:
            alleen_in_de_winkel = product.find_element(By.CSS_SELECTOR, '[data-testhook="product-availability-label"]')
            if "Alleen in de winkel" in alleen_in_de_winkel.text:
                product_name = product.find_element(By.CSS_SELECTOR, '[data-testhook="product-title"]').text
                product_link = product.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
                product_image = product.find_element(By.CSS_SELECTOR, 'img').get_attribute('src')
                product_price = product.find_element(By.CSS_SELECTOR, 'data-testhook="price-amount').text
                print(product_name)
                producten.append({'name': product_name, 'link': product_link, 'image': product_image, 'price': product_price})
                print(product_link)
        except NoSuchElementException:
            continue
    
    driver.quit()
    return producten








In [22]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

all_products = []
product_list = extract_ah_products(driver, category_urls[1])
all_products.extend(product_list)

df = pd.DataFrame(product_list)

# Save the DataFrame to a CSV file
df.to_csv('producten.csv', index=False)

print(df.head())

AH Frikandelbroodje
https://www.ah.nl/producten/product/wi230720/ah-frikandelbroodje
AH Saucijzenbroodje
https://www.ah.nl/producten/product/wi230724/ah-saucijzenbroodje
AH Uramaki
https://www.ah.nl/producten/product/wi231693/ah-uramaki
AH Kip paprikabroodje
https://www.ah.nl/producten/product/wi410181/ah-kip-paprikabroodje
AH Kaas uienbroodje
https://www.ah.nl/producten/product/wi411164/ah-kaas-uienbroodje
AH Sushi cali
https://www.ah.nl/producten/product/wi415358/ah-sushi-cali
AH Focaccia pepperoni
https://www.ah.nl/producten/product/wi425912/ah-focaccia-pepperoni
AH Focaccia mozzarella
https://www.ah.nl/producten/product/wi451137/ah-focaccia-mozzarella
AH Frikandelbroodje vegan
https://www.ah.nl/producten/product/wi523680/ah-frikandelbroodje-vegan
AH Pizzastuk vegetarisch
https://www.ah.nl/producten/product/wi537022/ah-pizzastuk-vegetarisch
AH Mezzelune cacio e pepe
https://www.ah.nl/producten/product/wi552782/ah-mezzelune-cacio-e-pepe
AH Oven Varkensfilet sticky crumble
https://www

In [17]:
category_urls[5]

'https://www.ah.nl/producten/bakkerij-en-banket'

In [8]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

product_list = extract_ah_products(driver, category_urls[1])
all_products.extend(product_list)

AH Frikandelbroodje
https://www.ah.nl/producten/product/wi230720/ah-frikandelbroodje
AH Saucijzenbroodje
https://www.ah.nl/producten/product/wi230724/ah-saucijzenbroodje
AH Uramaki
https://www.ah.nl/producten/product/wi231693/ah-uramaki
AH Kip paprikabroodje
https://www.ah.nl/producten/product/wi410181/ah-kip-paprikabroodje
AH Kaas uienbroodje
https://www.ah.nl/producten/product/wi411164/ah-kaas-uienbroodje
AH Sushi cali
https://www.ah.nl/producten/product/wi415358/ah-sushi-cali
AH Focaccia pepperoni
https://www.ah.nl/producten/product/wi425912/ah-focaccia-pepperoni
AH Focaccia mozzarella
https://www.ah.nl/producten/product/wi451137/ah-focaccia-mozzarella
AH Frikandelbroodje vegan
https://www.ah.nl/producten/product/wi523680/ah-frikandelbroodje-vegan
AH Pizzastuk vegetarisch
https://www.ah.nl/producten/product/wi537022/ah-pizzastuk-vegetarisch
AH Mezzelune cacio e pepe
https://www.ah.nl/producten/product/wi552782/ah-mezzelune-cacio-e-pepe
AH Oven Varkensfilet sticky crumble
https://www

In [17]:
category_urls[3]

'https://www.ah.nl/producten/kaas-vleeswaren-tapas'

In [9]:
all_products

[{'name': 'AH Patentbloem',
  'link': 'https://www.ah.nl/producten/product/wi951/ah-patentbloem'},
 {'name': 'AH Stokbrood wit',
  'link': 'https://www.ah.nl/producten/product/wi32543/ah-stokbrood-wit'},
 {'name': 'AH Allinson volkoren',
  'link': 'https://www.ah.nl/producten/product/wi48715/ah-allinson-volkoren'},
 {'name': 'AH Les pains Boulogne half',
  'link': 'https://www.ah.nl/producten/product/wi125034/ah-les-pains-boulogne-half'},
 {'name': 'AH Les pains boulogne heel',
  'link': 'https://www.ah.nl/producten/product/wi125035/ah-les-pains-boulogne-heel'},
 {'name': 'AH Mini gedecoreerde crodot',
  'link': 'https://www.ah.nl/producten/product/wi126531/ah-mini-gedecoreerde-crodot'},
 {'name': 'AH Kaassnack',
  'link': 'https://www.ah.nl/producten/product/wi143648/ah-kaassnack'},
 {'name': 'AH Ongebrande garneer amandelen',
  'link': 'https://www.ah.nl/producten/product/wi162131/ah-ongebrande-garneer-amandelen'},
 {'name': 'AH Tijger bruin ovenvers heel',
  'link': 'https://www.ah.

In [13]:
df = pd.DataFrame(all_products)


,name,link
0,AH Patentbloem,https://www.ah.nl/producten/product/wi951/ah-p...
1,AH Stokbrood wit,https://www.ah.nl/producten/product/wi32543/ah...
2,AH Allinson volkoren,https://www.ah.nl/producten/product/wi48715/ah...
3,AH Les pains Boulogne half,https://www.ah.nl/producten/product/wi125034/a...
4,AH Les pains boulogne heel,https://www.ah.nl/producten/product/wi125035/a...
...,...,...
199,AH Focaccia mozzarella,https://www.ah.nl/producten/product/wi451137/a...
200,AH Frikandelbroodje vegan,https://www.ah.nl/producten/product/wi523680/a...
201,AH Pizzastuk vegetarisch,https://www.ah.nl/producten/product/wi537022/a...
202,AH Mezzelune cacio e pepe,https://www.ah.nl/producten/product/wi552782/a...
